In [0]:
emp_data = [(1,"manish","11-07-2023","10:20"),
        (1,"manish","11-07-2023","11:20"),
        (2,"rajesh","11-07-2023","11:20"),
        (1,"manish","11-07-2023","11:50"),
        (2,"rajesh","11-07-2023","13:20"),
        (1,"manish","11-07-2023","19:20"),
        (2,"rajesh","11-07-2023","17:20"),
        (1,"manish","12-07-2023","10:32"),
        (1,"manish","12-07-2023","12:20"),
        (3,"vikash","12-07-2023","09:12"),
        (1,"manish","12-07-2023","16:23"),
        (3,"vikash","12-07-2023","18:08")]

emp_schema = ["id", "name", "date", "time"]
emp_df = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
window = Window.partitionBy("id", "date").orderBy("time").rangeBetween(Window.unboundedPreceding, Window.unboundedFollowing)

In [0]:
emp_df = emp_df.withColumn("event_ts", 
                           to_timestamp(concat_ws(" ", col("date"), col("time")), "dd-MM-yyyy HH:mm:ss")
                           )

In [0]:
emp_df.withColumn("login", first("event_ts").over(window))\
    .withColumn("logout", last("event_ts").over(window))\
    .withColumn("login", to_timestamp("login", "yyyy-MM-dd HH:mm:ss"))\
    .withColumn("logout", to_timestamp("logout", "yyyy-MM-dd HH:mm:ss"))\
    .withColumn("total_time", col("logout") - col("login")).show()